In [ ]:
# move or copy files?
copy_or_move = 'copy'

In [1]:
import os

# set path to input director containing all files:
input_dir = 'D:/InsectSound1000'

# Set path to output directory:
base_dir = 'D:/InsectSound1000'


labels = ['Coccinella',
          'Episyrphus',
          'Bombus',
          'Rhaphigaster',
          'Bradysia',
          'Aphidoletes', 
          'Halyomorpha',
          'Nezara',
          'Palomena',
          'Trialeurodes',
          'Myzus',
          'Tuta']

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


In [2]:
# Create split directories:
os.mkdir(train_dir)
os.mkdir(validation_dir)
os.mkdir(test_dir)

In [8]:
import os
import shutil
from fetchfiles import fetchfiles

for label in labels:
    this_label_files = fetchfiles(input_dir, label)
    print(str(len(this_label_files)) + ' files found for ' + label)

    # get all recording dates:
    dates = []
    for file in this_label_files:
        _, fname = os.path.split(file)
        date = fname[:8]
        if date not in dates:
            dates.append(date)

    # use list compression to build a dates dictionary for this label:
    dates_dict = {}
    for date in dates:
        dates_dict[date] = [file for file in this_label_files if date in file]
        
    # sort by lenght of list:
    dates_dict = dict(sorted(dates_dict.items(), key=lambda item: len(item[1]), reverse=True))
    # print sorted dict:
    for key, value in dates_dict.items():
        print(key, len(value))
    
    # now let's split the files into train, validation and test sets:
    # make sure dates don't overlap between sets:
    train_files = []
    validation_files = []
    test_files = []
    
    if len(dates_dict) < 3:
        print('Not enough dates to split the data into train, validation and test sets.')
        break
    else:
        # biggest date goes to train, secound biggest to validation, thrid biggest to test
        # rest gets added to train:
         # put the big files in frist:
        train_files.extend(dates_dict.pop(next(iter(dates_dict))))
        validation_files.extend(dates_dict.pop(next(iter(dates_dict))))
        test_files.extend(dates_dict.pop(next(iter(dates_dict))))
        
        while len(dates_dict) > 0:
        
            # but, make sure we have at least 1000 sample in the test set:
            while(len(test_files)/len(this_label_files)) < 0.15:
                if len(dates_dict) == 0:
                    break
                # add the smallest date to test set:
                test_files.extend(dates_dict.pop(next(reversed(dates_dict))))
                
            # but, make sure validation set is not to small:
            while (len(validation_files)/len(this_label_files)) < 0.15:
                if len(dates_dict) == 0:
                    break
                # add the smallest left date to validation set:
                validation_files.extend(dates_dict.pop(next(reversed(dates_dict))))
            
            # add the rest to train set:
            if len(dates_dict) > 0:
                train_files.extend(dates_dict.pop(next(reversed(dates_dict))))
        
    print('%s %s %s' % (str(len(train_files)), str(len(validation_files)), str(len(test_files))))
    print(str(round(len(train_files)/len(this_label_files)*100, 2)) + '% ' 
          + str(round(len(validation_files)/len(this_label_files)*100, 2)) + '% ' 
          + str(round(len(test_files)/len(this_label_files)*100, 2)) + '%')
    print()  

    # now let's copy the files to the new directories:
    for files, dst_folder in zip([train_files, validation_files, test_files], [train_dir, validation_dir, test_dir]):
        for file in files:
            dontneedthis, fname = os.path.split(file)
            src = file
            dst = os.path.join(dst_folder, fname)
            if copy_or_move == 'move':
                shutil.move(src, dst)
            elif copy_or_move == 'copy':
                shutil.copy(src, dst)     
       

224 files found for Coccinella
20230418 70
20230420 68
20230419 48
20230425 26
20230424 12
108 68 48
48.21% 30.36% 21.43%

868 files found for Episyrphus
20230515 653
20230612 155
20230511 60
653 155 60
75.23% 17.86% 6.91%

6405 files found for Bombus
20230510 2353
20230427 2069
20230509 1983
2353 2069 1983
36.74% 32.3% 30.96%

733 files found for Rhaphigaster
20230925 211
20230920 184
20231004 123
20230918 103
20231005 58
20230919 48
20230914 6
426 184 123
58.12% 25.1% 16.78%

113 files found for Bradysia
20220329 57
20220331 37
20220330 19
57 37 19
50.44% 32.74% 16.81%

267 files found for Aphidoletes
20220504 110
20220503 67
20220324 33
20220502 31
20220323 26
141 67 59
52.81% 25.09% 22.1%

735 files found for Halyomorpha
20220601 215
20220530 160
20220531 108
20220421 104
20220420 64
20220524 47
20220425 37
430 160 145
58.5% 21.77% 19.73%

896 files found for Nezara
20220511 681
20220512 90
20220602 65
20220614 32
20220523 28
681 90 125
76.0% 10.04% 13.95%

1282 files found for Pal